
... ***CURRENTLY UNDER DEVELOPMENT*** ...


# Nearshore storms (SWAN+snell) at 36 profiles to hourly data

In [7]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.climate_emulator import Climate_Emulator
from teslakit.waves import AWL, Aggregate_WavesFamilies, Intradaily_Hydrograph
from teslakit.util.time_operations import xds_limit_dates, repair_times_hourly, add_max_storms_mask


In [8]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'

# climate change - S4
site = 'KWAJALEIN_CC_S4'

db_S4 = Database(p_data)
db_S4.SetSite(site)

# Fichero AlbaR
nc_nearshore_storms = '/Users/albacid/Projects/SERDP/inputs_Kwajalein/waves_nearshore_emulador/RCM_TCs_profiles_CC_S4.nc'


## Load offshore waves

In [3]:
# Full simulation

CE = Climate_Emulator(db_S4.paths.site.EXTREMES.climate_emulator)

WVS_upd = CE.LoadSim_All(TCs=True)
WVS_upd = WVS_upd.to_xarray()

# set time as coordinate
WVS_upd = WVS_upd.swap_dims({'index': 'time'})

print(WVS_upd)


<xarray.Dataset>
Dimensions:      (time: 1032644)
Coordinates:
    index        (time) int64 0 1 2 3 4 ... 1032640 1032641 1032642 1032643
  * time         (time) object 2000-01-01 00:00:00 ... 3000-01-01 00:00:00
Data variables:
    DWT          (time) float32 28.0 1.0 32.0 33.0 41.0 ... 13.0 28.0 20.0 20.0
    sea_Hs       (time) float32 1.5582333 2.434917 ... 1.810582 2.2550507
    sea_Tp       (time) float32 7.720595 8.870701 ... 8.210051 6.6658435
    sea_Dir      (time) float32 54.23647 70.633 ... 56.327915 67.612114
    swell_1_Hs   (time) float32 0.84767956 0.86827296 ... 0.3111764 0.42345622
    swell_1_Tp   (time) float32 8.847652 6.94969 ... 9.856896 11.085974
    swell_1_Dir  (time) float32 68.4031 97.40611 45.76165 ... 73.13391 72.43033
    swell_2_Hs   (time) float32 0.35236874 0.28442293 ... 0.45500863 0.26068825
    swell_2_Tp   (time) float32 11.0032215 12.334754 ... 11.017166 9.1800585
    swell_2_Dir  (time) float32 179.73355 171.34354 ... 179.41365 171.41524
    win

In [4]:
# DatetimeGregorian to datetime
#time = [x.timetuple() for x in WVS_upd.time.values];
#time = [datetime(*x[:6]) for x in time];

#WVS_upd.time.values = time


## Load nearshore waves 

In [5]:
# Full simulation

RCM_TCs_profiles = xr.open_dataset(nc_nearshore_storms)
RCM_TCs_profiles = RCM_TCs_profiles.drop({'tm', 'dirP'})
RCM_TCs_profiles = RCM_TCs_profiles.rename({'hs':'Hs', 'dir':'Dir', 'tp':'Tp'})
print(RCM_TCs_profiles)


<xarray.Dataset>
Dimensions:  (index: 1032644, profile: 36)
Coordinates:
  * index    (index) float64 0.0 1.0 2.0 3.0 ... 1.033e+06 1.033e+06 1.033e+06
  * profile  (profile) float64 1.0 2.0 3.0 4.0 5.0 ... 32.0 33.0 34.0 35.0 36.0
Data variables:
    Hs       (index, profile) float64 ...
    Tp       (index, profile) float64 ...
    Dir      (index, profile) float64 ...


## Storms to hourly

In [6]:
# Iterate over simulations
for n in range(7,10):
    
    # select simulation offshore
    WVS_upd_n = WVS_upd.where(WVS_upd.n_sim==n, drop=True).copy(deep=True)
    
    # select simulation nearshore
    RCM_TCs_profiles_n = RCM_TCs_profiles.sel(index=WVS_upd_n.index).copy(deep=True)
    
    # keep needed parameters
    mu_tau_ss = WVS_upd_n[['mu','tau','ss','time']].copy(deep=True)
    
    # add time variable
    RCM_TCs_profiles_n['time'] = WVS_upd_n.time.copy(deep=True)
    
    
    # Iterate over profiles
    for p in RCM_TCs_profiles.profile:

        # select profile nearshore
        RCM_TCs_profiles_p = RCM_TCs_profiles_n.sel(profile=p).copy(deep=True)
        
        # calculate intradaily hourly hydrographs for simulated storms
        WVS_h = Intradaily_Hydrograph(RCM_TCs_profiles_p, mu_tau_ss)

        # calculate AWL
        WVS_h['AWL'] = AWL(WVS_h.Hs, WVS_h.Tp)
        
        # repair times: remove duplicates (if any)
        WVS_h = repair_times_hourly(WVS_h)

        # add profile
        WVS_h = WVS_h.assign_coords(profile=RCM_TCs_profiles_p.profile).expand_dims('profile')
                
        # save profiles
        if p==1:            
            WVS_h_out = WVS_h.copy(deep=True)
        else:
            WVS_h_out = xr.concat([WVS_h_out, WVS_h], dim='profile')

    

    # store hourly simulation nearshore data
    db_S4.Save_SIM_NEARSHORE(WVS_h_out, n)

    print('simulation {0} processed.'.format(n))    


simulation 7 processed.
simulation 8 processed.
simulation 9 processed.
